# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


In [ ]:
import pandas as pd 
import os 
import sys

+ Load the environment variables using dotenv. (1 pt)

In [ ]:
# Write your code below.
%load_ext dotenv
%dotenv 


In [ ]:
sys.path.append(os.getenv('SRC_DIR'))

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [ ]:
PRICE_DATA = os.getenv("PRICE_DATA")
PRICE_DATA

In [ ]:
import os
from glob import glob

# Write your code below.
parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet"))

In [ ]:
import dask.dataframe as dd
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
# Write your code below.
# Define a function that will be applied to each group (?)
def add_features(group):
    # Create lagged versions of Close and Adj_Close
    group['Close_lag'] = group['Close'].shift(1)
    group['Adj_Close_lag'] = group['Adj_Close'].shift(1)

    # Calculate returns based on Adjusted Close
    group['returns'] = (group['Adj_Close'] / group['Adj_Close_lag']) - 1

    # Calculate high-low range for the day
    group['hi_lo_range'] = group['High'] - group['Low']
    
    return group

# Apply the function to each group (ticker) using groupby
dd_feat = dd_px.groupby('ticker', group_keys=False).apply(lambda x: add_features(x))


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [ ]:
# Write your code below.
dd_feat_pd = dd_feat.compute()


In [ ]:
dd_feat_pd = dd_feat_pd.sort_values(by=['ticker', 'date'])
dd_feat_pd['rolling_avg_returns'] = dd_feat_pd.groupby('ticker')['returns'].rolling(window=10, min_periods=1).mean().reset_index(level=0, drop=True)

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

I think the computation would have been faster with Dask. Probably not a huge noticeable difference, but faster regardless. 

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.